#Import data

- I used below import the XMl file
- Tryed to import straight from my google drive, bt for som reason it did not seem to store the correct file in memory. Son instead I used upload code
 - More information about how to import files into colab can be found here: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=c2W5A2px3doP

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Importing and parsing the XML-file 

In [0]:
# Importing packages
import xml.etree.ElementTree

#Parsing XML file and getting root
XML_parsed = xml.etree.ElementTree.parse('00001 (1).xml')
root=XML_parsed.getroot()

In [15]:
#Example of how to get the values from a tag. Below we get the values from the <mnemonicList> tag
root.find(".//{http://www.witsml.org/schemas/1series}mnemonicList").text

'TIME,SRVTVD,GS_SPM1,GS_TV11,SRV_HY,HKLD30s,UTSTAT,SWOB30s,GS_TV03,SPPA,ETSL,AJAM_MWD,ATMP_RT,PMIN_TM_RT,HKLD,GS_G_C1,GS_TVCA,DNI_MP,BPOS,GS_DVER,HKLN,GS_TV06,GS_TV05,SGX,GS_SWOB,HZ_RT,PMAX_TM_RT,GS_DRTV,CRS_CNAZI,DBTV,CUREDT_RT,GS_G_C2,STK2,SHK2_RT,APRS_RT,TTONB,GS_TV01,SHX,GS_ACTC,SGZ,PMIN_RT,SRVEW,INCL_CONT_RT,SPM1,GS_TV13,BHFG,BVEL,TFLO,TOTAL_CORR,GS_G_NC5,MSP4,PESD_TM_RT,TOJ,ACTC,CPOF,RSUX,HKLX,STK1,EDRT,GS_DCHM,GS_TV04,ECD_MW_IN,TEMP_DNI_RT,GS_TV08,NRPM_RT,THKD,PASS_NAME,TOFF,CRS4,GS_DMEA,SRV_HX,SRVAZI,GS_G_NC4,TEFNEG,A40H_UNC_RT,ANGLX,SGY,SDEP_CONT_RT,GS_CFIA,GS_DBTM,GS_DRTM,TCHR,HKLI,ESD_RT,SRV_GX,GS_CHKP,SHY,GS_HKLD,RHX,MBOT,FVOC,SHKTOT_RT,HY_RT,GS_MTOA,ESD_MAX_RT,ROP,A40H_COND_RT,STWT,PMPT,GS_SPM2,P40H_COND_RT,RGX_RT,SRVNS,GS_BPOS,PMAX_RT,A28H_COND_RT,STUCK_RT,DCHR,DNISTAT,CRPM_RT,DBTM,GRID_CORR,STK4,SRV_HZ,ROP5,CRS1,ARC_GR_UNC_RT,GS_TV10,MAG_DEC,QROP,TOFB,SPM3,AZIM_CONT_RT,TVDE,OSTM,TSTK,SSAMP,GS_SPM3,ESD_MIN_RT,RSDX,SRV_GZ,GS_TV02,BITRUN,HYDR_RET_P,Stick_RT,PESD_RT,MTF_RT,R

Pupulating a dataframe with elements from the parsed XML file

In [19]:
# Importing packages
import numpy as np
import pandas as pd
from datetime import datetime

#Fetching the elements from the MnemonicList tag (log short-names) into a list
mnemonicList=root.find(".//{http://www.witsml.org/schemas/1series}mnemonicList").text
mnemonicList=mnemonicList.split(",")

#Fetching the elements from the UnitList tag (unirts for the data) into a list
unitList=root.find(".//{http://www.witsml.org/schemas/1series}unitList").text
unitList.split(",")

#Fetching the elements from the Data tag into a list
data=[]
for m in root.findall(".//{http://www.witsml.org/schemas/1series}data"):
    string=m.text
    data.append(string.split(","))
    
#Converting list of data elements to an numpy array (of strings)
log_data_array=np.asarray(data)

#Converting numpy array to a DataFrame with the element from MnemonicList as column names
df_log=pd.DataFrame(log_data_array, columns=mnemonicList)

#Replacing empty string elements to NaN
df_log=df_log.replace('', np.nan)

#All the numbers in the DataFrame are stored as strings. These elements should be numbers, with exception of the PASS_NAME (comments) and TIME columns
#Converting all elements to float64, except the PASS_NAME column
for key in df_log.keys():
    if key=='PASS_NAME':
        df_log[key]=df_log[key].astype('object')
    else:
      if key=='TIME':
        df_log[key]=df_log[key].astype('datetime64')
      else:
        df_log[key]=df_log[key].astype('float64')

#Setting the time column as index
df_log=df_log.set_index('TIME')

/usr/local/lib/python3.6/dist-packages/pandas/core/internals.py:3462: FutureWarning: Passing in 'datetime64' dtype with no frequency is deprecated and will raise in a future version. Please pass in 'datetime64[ns]' instead.
  return self.apply('astype', dtype=dtype, **kwargs)


The relevant time series data should now be populated in the DataFrame df_log

In [22]:
df_log.head()

,SRVTVD,GS_SPM1,GS_TV11,SRV_HY,HKLD30s,UTSTAT,SWOB30s,GS_TV03,SPPA,ETSL,...,RPM,STKSLP,BP_CORR_P,SHKLV,GS_MDOA,SSFRQ,SPM4,RGX,GTF_RT,GS_G_IC4
TIME,,,,,,,,,,,,,,,,,,,,,
2009-01-14 05:39:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.36,NaN,NaN,NaN,NaN,NaN
2009-01-17 06:03:05,NaN,0.0,52.980000,NaN,NaN,NaN,NaN,0.37,NaN,NaN,...,NaN,NaN,NaN,NaN,0.82,NaN,NaN,NaN,NaN,NaN
2009-01-17 06:03:09,NaN,0.0,52.950001,NaN,NaN,NaN,NaN,0.37,NaN,NaN,...,NaN,NaN,NaN,NaN,0.82,NaN,NaN,NaN,NaN,NaN
2009-01-17 06:03:14,NaN,0.0,52.939999,NaN,NaN,NaN,NaN,0.37,NaN,NaN,...,NaN,NaN,NaN,NaN,0.82,NaN,NaN,NaN,NaN,NaN
2009-01-17 06:03:18,NaN,0.0,52.919998,NaN,NaN,NaN,NaN,0.37,NaN,NaN,...,NaN,NaN,NaN,NaN,0.82,NaN,NaN,NaN,NaN,NaN
